In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-24 19:38:04--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-02-24 19:38:05 (5.99 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read watch reviews 
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep=r'\t', header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [5]:
# Count Rows in Dataframe
df.count()

1801974

In [6]:
# Build reviews dataframe
columns = ["review_id","customer_id","product_id","product_parent","review_date"]
reviews_df = df.select(columns)
reviews_df = reviews_df.withColumn("customer_id", reviews_df["customer_id"].cast(IntegerType()))
reviews_df = reviews_df.withColumn("product_parent", reviews_df["product_parent"].cast(IntegerType()))
reviews_df = reviews_df.withColumn("review_date",to_date(col("review_date"),"yyyy-mm-dd"))
reviews_df.show()
reviews_df.count()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1NBG94582SJE2|    2975964|B00I01JQJM|     860486164| 2015-01-31|
|R273DCA6Y0H9V7|   23526356|B00TCO0ZAA|     292641483| 2015-01-31|
| RQVOXO7WUOFK6|   52764145|B00B7733E0|      75825744| 2015-01-31|
|R1KWKSF21PO6HO|   47348933|B006ZN4U34|     789352955| 2015-01-31|
|R38H3UO1J190GI|   33680700|B00HUEBGMU|      19067902| 2015-01-31|
|R3NPIFKLR19NQA|   30301059|B008MW6Y12|     597683407| 2015-01-31|
|R3MBE6UCH3435E|   28282645|B00TE8XKIS|      35563334| 2015-01-31|
|R2E7A4FF0PVY5Q|     502818|B00ZKDUFBQ|     555190742| 2015-01-31|
| R3R8JDQ2BF4NM|    1481233|B010BZ7S2Q|     129544315| 2015-01-31|
|R1YND4BS823GN5|   27885926|B00HRXSSRA|     708418657| 2015-01-31|
|R2TZNSA18V7YF6|    3183883|B005C95NM4|     246957815| 2015-01-31|
|R22ZVRDPPXIDNL|   23208852|B00LBIBYBE|     746593019| 2015-01

1801974

In [7]:
reviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [8]:
# Build products dataframe
columns = ["product_id","product_title"]
products_df = df.select(columns)
products_df = products_df.dropDuplicates()
products_df.show()
products_df.count()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00L9ZF5BY|Flylinktech Actio...|
|B00017LSF8|B+W Clear UV Haze...|
|B00CGTN2N0|Queen Spy Mini U8...|
|B002POMZWU|Usb cable for kod...|
|B00FEY2BCW|Camera Case Eveca...|
|B0131XNIDS|*5-pack* microSue...|
|B004OUMTT2|Celestron PowerSe...|
|B004HY3XOA|58MM UV Lens Filt...|
|B00LBFKLRU|(Create a generic...|
|B007QXZ6D0|Vello Cinch Strap...|
|B00PG36JTW|Tamrac TEK Fitted...|
|B003JNJDGM|TTL Off Camera Sh...|
|B00ZYATKCA|Olympus Air Mirro...|
|B00S2X48IW|Qkoo Selfie Handh...|
|B00VQJASPG|    Canon SL1 Bundle|
|B00554PCDG|Pixel Flash Hot S...|
|B00006JPWW|Panasonic AY-DVMC...|
|B00B9Z23NM|Nikon 8250 ACULON...|
|B00GOJ7JGY|Fotasy HET63 Lens...|
|B00IUU22WG|White Photography...|
+----------+--------------------+
only showing top 20 rows



168675

In [9]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [10]:
# Build customers dataframe
customers_df = df.groupby("customer_id").count()
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df = customers_df.withColumn("customer_id", customers_df["customer_id"].cast(IntegerType()))
customers_df = customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))
customers_df.show()
customers_df.count()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     109519|             3|
|   45046643|             2|
|   45676353|            29|
|   49258595|             1|
|   11081770|             1|
|   51671883|            10|
|   23678637|             1|
|   14479333|             2|
|   12318800|             1|
|   41257827|             3|
|   52809621|             2|
|   31693763|             1|
|   52942372|             4|
|   10820140|             1|
|   45253663|             4|
|    3910358|             1|
|   43009502|             4|
|   52809622|             5|
|   14781308|             1|
|   27301220|             9|
+-----------+--------------+
only showing top 20 rows



1116762

In [11]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [12]:
# Configure settings for RDS
# Note: To test this code, replace the following:
# <rds connection string/url>
# <rds db name>
# <userid>
# <password>
mode = "append"
jdbc_url="jdbc:postgresql://<rds connection string/url>:5432/<rds db name>"
config = {"user":"<userid>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [14]:
# write customers to table
customers_df.write.jdbc(url=jdbc_url, table='customers2', mode=mode, properties=config)

In [15]:
# Write products to table
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [16]:
# Write reviews to table
reviews_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)